In [1]:
import psycopg2
import pandas as pd
import psycopg2.extras
 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler, normalize

from sklearn.linear_model import  LinearRegression

In [2]:
class PostgresConnection(object):
    def __init__(self):
        self.connection = psycopg2.connect(database="ecomdb",
                                           user = "postgres",
                                           password = "123456",
                                           host = "127.0.0.1",
                                           port = "5432")

    def getConnection(self):
        print("successfully connected to database")
        return self.connection
    
    
# con = PostgresConnection().getConnection() 

In [3]:
#  select_stmt= is the SQL select query along with  table and column name    
def retrive_data_from_table(select_stmt): 
    con = PostgresConnection().getConnection()
    cur = con.cursor()
    #select_stmt = select_stmt = "SELECT * FROM :{} ".formate(tablename)
    cur.execute(select_stmt)
    records = cur.fetchall()
    return records

In [5]:
 
select_stmt = '''select  t.year,
               t.month,
               i.item_name,
               sum(f.total_price) 
               from ecomdb_star_schema.fact_table f 
               join ecomdb_star_schema.time_dim t on t.time_key = f.time_key
               join ecomdb_star_schema.item_dim i on i.item_key = f.item_key
               group by (i.item_name,t.year,t.month)
               order by (i.item_name,t.year) '''
 

records = retrive_data_from_table(select_stmt)
df = pd.DataFrame(list(records), columns=['year', 'month', 'item_name', 'Total Sales'])

df.head()

successfully connected to database


,year,month,item_name,Total Sales
0,2014,1,100% Juice Box Variety 6.75 oz,1980.0
1,2014,2,100% Juice Box Variety 6.75 oz,3360.0
2,2014,3,100% Juice Box Variety 6.75 oz,4470.0
3,2014,4,100% Juice Box Variety 6.75 oz,3060.0
4,2014,5,100% Juice Box Variety 6.75 oz,4500.0


In [7]:
X = df.iloc[:,:-1]
# X.item_name.duplicated().sum()
X

,year,month,item_name
0,2014,1,100% Juice Box Variety 6.75 oz
1,2014,2,100% Juice Box Variety 6.75 oz
2,2014,3,100% Juice Box Variety 6.75 oz
3,2014,4,100% Juice Box Variety 6.75 oz
4,2014,5,100% Juice Box Variety 6.75 oz
...,...,...,...
22010,2020,9,Zoo Animal Cookies/Crackers
22011,2020,10,Zoo Animal Cookies/Crackers
22012,2020,11,Zoo Animal Cookies/Crackers
22013,2020,12,Zoo Animal Cookies/Crackers


In [12]:
X.dtypes

year                                               int64
month                                              int64
item_name_100% Juice Box Variety 6.75 oz           uint8
item_name_A&W Root Beer - 12 oz cans               uint8
item_name_A&W Root Beer Diet - 12 oz cans          uint8
                                                   ...  
item_name_Welch's 100% Juice Variety10 oz          uint8
item_name_Wint O Green LifeSavers Sugar Free       uint8
item_name_Wonderful Pistachios (in shell)          uint8
item_name_Yoplait Original Yogurt Variety Pack     uint8
item_name_Zoo Animal Cookies/Crackers              uint8
Length: 261, dtype: object

In [13]:
# df3 = pd.get_dummies(df2, columns = ['Month', 'year'])
# X = pd.get_dummies(X,columns = ['item_name'])
# X = pd.get_dummies(X, columns = ['item_name'])

X.head()

,year,month,item_name_100% Juice Box Variety 6.75 oz,item_name_A&W Root Beer - 12 oz cans,item_name_A&W Root Beer Diet - 12 oz cans,item_name_Advil 2 pill packets,item_name_All Purpose Cleaner 409 Spray,item_name_Altoids Small Packs Variety,item_name_Apple Sauce Variety GoGO Squeeze 3.2oz,item_name_AquaFina Purified Drinking Water 16oz,...,item_name_Waterloo Sparkling Lemon 12oz,item_name_Waterloo Sparkling Lime 12oz,item_name_Waterloo Sparkling Mango 12oz,item_name_Waterloo Sparkling Original 12oz,item_name_Waterloo Sparkling Watermelon 12oz,item_name_Welch's 100% Juice Variety10 oz,item_name_Wint O Green LifeSavers Sugar Free,item_name_Wonderful Pistachios (in shell),item_name_Yoplait Original Yogurt Variety Pack,item_name_Zoo Animal Cookies/Crackers
0,2014,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2014,2,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2014,3,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2014,4,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2014,5,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
# X.drop('year', axis= 2, inplace = True)
X = X.drop(['year'], axis=1)
X = X.drop(['month'], axis=1)

X.head()

,item_name_100% Juice Box Variety 6.75 oz,item_name_A&W Root Beer - 12 oz cans,item_name_A&W Root Beer Diet - 12 oz cans,item_name_Advil 2 pill packets,item_name_All Purpose Cleaner 409 Spray,item_name_Altoids Small Packs Variety,item_name_Apple Sauce Variety GoGO Squeeze 3.2oz,item_name_AquaFina Purified Drinking Water 16oz,item_name_Austin Cookie Cracker Variety,item_name_Austin Toasty Crackers w/ PBtr,...,item_name_Waterloo Sparkling Lemon 12oz,item_name_Waterloo Sparkling Lime 12oz,item_name_Waterloo Sparkling Mango 12oz,item_name_Waterloo Sparkling Original 12oz,item_name_Waterloo Sparkling Watermelon 12oz,item_name_Welch's 100% Juice Variety10 oz,item_name_Wint O Green LifeSavers Sugar Free,item_name_Wonderful Pistachios (in shell),item_name_Yoplait Original Yogurt Variety Pack,item_name_Zoo Animal Cookies/Crackers
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
y = df.iloc[:,-1]
y.head()

0    1980.0
1    3360.0
2    4470.0
3    3060.0
4    4500.0
Name: Total Sales, dtype: object

In [19]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.2,random_state = 10)

In [21]:
regressor = LinearRegression()

In [22]:
regressor.fit(X_train,y_train)

LinearRegression()

In [24]:
pred = regressor.predict(X_test)
pred

array([3754.   , 4253.5  , 4399.   , ..., 3200.25 , 3113.125, 5792.5  ])

In [23]:
regressor.score(X_test,y_test)

0.8510074157967907